In [1]:
import os, sys                                  # system functions
import nipype.interfaces.io as nio           # Data i/o
from nipype.interfaces.io import DataSink
import nipype.interfaces.fsl as fsl          # fsl
import nipype.pipeline.engine as pe          # pypeline engine
import nipype.interfaces.utility as util     # utility
import nipype.algorithms.modelgen as model   # model generation
import errno
import glob
import pandas as pd

fsl.FSLCommand.set_default_output_type('NIFTI_GZ')

### Copes at level 2:
##### Model 1:
conditions = 
- 'cuestimrespfeedback',
- 'response_leftminright',
- 'cue_SPDminACC',
- 'stimulus_value_difference',
- 'feedback_PE'


In [2]:
# general set-up
project_folder = '/home/Public/trondheim'
work_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
# The directory where we can find the level 2 output
slm_folder = os.path.join(project_folder, 'derivatives', 'glm_feat', 'subject_level_model')


spaces = ['T1w']   # shouldn't touch this but just in case we _do_ want to go back to MNI....
ses = 'rlsat'      # don't make this a list, that won't work
task = 'rlsat'     # don't make this a list, that won't work
model_ns = ['1-dm-RTdur']      # 
fwhms = ['1p5']

template_brain = '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii'
template_brain_mask = '/home/Public/trondheim/sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii'

subject_ids = sorted(glob.glob(os.path.join(project_folder, 'derivatives', 'glm_feat', 'subject_level_model', 'sub-*', 'ses-rlsat', 'func', 'fwhm-1p5', 'model-{}'.format(model_ns[0]))))
subject_ids = [x.split('/')[-5].split('-')[-1] for x in subject_ids]

## TMP
#subject_ids = subject_ids[:-1]

contrasts = ['0','1','2','3','4', '5']  # from second-level model

templates = {'level2_cope': os.path.join(slm_folder, 'sub-*', 'ses-rlsat', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-rlsat_task-rlsat_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-cope.nii.gz'),
             'level2_varcope': os.path.join(slm_folder, 'sub-*', 'ses-rlsat', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-rlsat_task-rlsat_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-varcope.nii.gz'),
             'level2_tdof': os.path.join(slm_folder, 'sub-*', 'ses-rlsat', 'func', 'fwhm-{fwhm}', 'model-{model_n}', '*ses-rlsat_task-rlsat_space-MNI152NLin2009cAsym_model-{model_n}_contrast-{contrast_n}_desc-tdof_t.nii.gz')}

In [3]:
# ## make brain mask for flameo
# import nilearn
# from nilearn import plotting
# import nibabel as nib

# hdr = nib.load('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain.nii')
# brain_data = hdr.get_fdata()
# brain_data[brain_data>0] = 1
# brain_mask = nib.Nifti1Image(brain_data, affine=hdr.affine, header=hdr.header)
# brain_mask.to_filename('../sourcedata/templates/mni_icbm152_t1_tal_nlin_asym_09c_brain_mask.nii')

In [3]:
##
workflow = pe.Workflow(name='feat_level3_rlsat') ## TMP
workflow.base_dir = os.path.join(project_folder, 'processing', 'nipype_workflow_folders')
workflow.config = {"execution": {"crashdump_dir":os.path.join(project_folder, 'processing', 'crashdumps')}}

# Identity
identity = pe.Node(util.IdentityInterface(fields=['contrast_n', 'model_n', 'fwhm']), name='identity')
identity.iterables = [('contrast_n', contrasts),
                      ('fwhm', fwhms),
                      ('model_n', model_ns)]

# Selector
selector = pe.Node(nio.SelectFiles(templates), name='selector')
workflow.connect(identity, 'contrast_n', selector, 'contrast_n')
workflow.connect(identity, 'fwhm', selector, 'fwhm')
workflow.connect(identity, 'model_n', selector, 'model_n')

## Merge copes, varcopes, masks
copemerge    = pe.Node(interface=fsl.Merge(dimension='t'),
                          name="copemerge")

varcopemerge = pe.Node(interface=fsl.Merge(dimension='t'),
                       name="varcopemerge")

workflow.connect(selector, 'level2_cope', copemerge, 'in_files')
workflow.connect(selector, 'level2_varcope', varcopemerge, 'in_files')

In [4]:
# pd.DataFrame(get_design_matrix('0', get_subject_ids())[1])

In [5]:
def get_subject_ids():
    import glob
    import os
    subject_ids = sorted(glob.glob(os.path.join('/home/Public/trondheim', 'derivatives', 'glm_feat', 'subject_level_model', 'sub-*', 'ses-rlsat')))
    subject_ids = [x.split('/')[-2].split('-')[-1] for x in subject_ids]
    
    return subject_ids

subject_id_getter = pe.Node(util.Function(output_names=['subject_ids'],
                                          function=get_subject_ids),
                            name='subject_id_getter')

def get_design_matrix(second_level_contrast, subject_ids):
    print(second_level_contrast)
    regressors = {'intercept': [1 for x in range(len(subject_ids))]}
    
    if second_level_contrast == '0':  # SPD-ACC
        import pandas as pd
        # find behavioral model parameters
        parameters = pd.read_csv('/home/Public/trondheim/derivatives/behavior/task-rlsat_model-RLARD-v0B_group_parameters.tsv', sep='\t')
        for colname in ['v0_cues-a', 'B_cues-a']:
            parameters[colname+'_z'] = (parameters[colname]-parameters[colname].mean())/parameters[colname].std()
        
        parameters['subject_label'] = parameters['subjects'].apply(lambda x: str(x).zfill(3))
        thresholds = [parameters.loc[parameters.subject_label==x, 'B_cues-a_z'].values[0].round(5) for x in subject_ids]
        urgencies = [parameters.loc[parameters.subject_label==x, 'v0_cues-a_z'].values[0].round(5) for x in subject_ids]
        regressors['threshold'] = thresholds
        regressors['urgency'] = urgencies
#         parameters = pd.read_csv('/home/Public/trondheim/scripts/RLDMC/diagnostics/model-arw-RL-mag-SAT-B2_data-rlsat/medianParametersBPICs.csv').rename(columns={'Unnamed: 0': 'sub'}).iloc[:-2]
#         parameters['B0.spd'] = parameters['B0']*(1+parameters['B0Mod.SPD'])
#         parameters['threshold_z'] = parameters['B0']-parameters['B0.spd']
#         parameters['threshold_z'] = (parameters['threshold_z'] - parameters['threshold_z'].mean()) / (parameters['threshold_z'].std())
#         # parameters['threshold_z'] = (parameters['B0Mod.SPD'] - parameters['B0Mod.SPD'].mean())/parameters['B0Mod.SPD'].std()
#         # parameters['threshold_z'] = (parameters['B0Mod.SPD'] - parameters['B0Mod.SPD'].mean())/parameters['B0Mod.SPD'].std()
#         parameters['subject_label'] = parameters['sub'].apply(lambda x: str(x).zfill(3))
#         thresholds = [parameters.loc[parameters.subject_label==x, 'threshold_z'].values[0].round(5) for x in subject_ids]
    
#         regressors['threshold'] = thresholds

        # contrasts (3rd-level), include model-based covariate
        third_level_contrasts = [('Group mean', 'T', ['intercept', 'threshold', 'urgency'], [1.0, 0, 0]),
                                 ('-Group mean', 'T', ['intercept', 'threshold', 'urgency'], [-1.0, 0, 0]),
                                 ('Threshold', 'T', ['intercept', 'threshold', 'urgency'], [0, 1, 0]),
                                 ('-Threshold', 'T', ['intercept', 'threshold', 'urgency'], [0, -1, 0]),
                                 ('Urgency', 'T', ['intercept', 'threshold', 'urgency'], [0, 0, 1]),
                                 ('-Urgency', 'T', ['intercept', 'threshold', 'urgency'], [0, 0, -1]),
                                 ]
    else:
        # contrasts (3rd-level)
        third_level_contrasts = [('Group mean', 'T', ['intercept'], [1.0]),
                                 ('-Group mean', 'T', ['intercept'], [-1.0])
                                 ]
    
    return third_level_contrasts, regressors



contrastgen_l3 = pe.Node(util.Function(input_names=['second_level_contrast', 'subject_ids'],
                                       output_names=['third_level_contrasts', 'regressors'],
                                       function=get_design_matrix),
                      name='contrastgen_l3')

level3model = pe.Node(interface=fsl.MultipleRegressDesign(),
                      name='l3model')


workflow.connect(subject_id_getter, 'subject_ids', contrastgen_l3, 'subject_ids')
workflow.connect(identity, 'contrast_n', contrastgen_l3, 'second_level_contrast')
workflow.connect(contrastgen_l3, 'third_level_contrasts', level3model, 'contrasts')
workflow.connect(contrastgen_l3, 'regressors', level3model, 'regressors')

In [6]:
flameo = pe.Node(
    interface=fsl.FLAMEO(),
    name="flameo")

flameo.iterables = ('run_mode', ['flame1', 'flame12'])
flameo.inputs.mask_file = template_brain_mask
flameo.inputs.infer_outliers = False # True   # run with automatic outlier detection ## Seems to crash for contrast 3 (value)

workflow.connect([
    (copemerge, flameo, [('merged_file', 'cope_file')]),
    (varcopemerge, flameo, [('merged_file', 'var_cope_file')]),
    (level3model, flameo, [('design_mat', 'design_file'),
                           ('design_con', 't_con_file'), 
                           ('design_grp', 'cov_split_file')]),
])

In [7]:
## cluster thresholding
# Smoothness estimation
smoothestimate = pe.MapNode(fsl.SmoothEstimate(), iterfield=['zstat_file'], name='smoothestimate')
smoothestimate.inputs.mask_file = template_brain_mask

workflow.connect(flameo, 'zstats', smoothestimate, 'zstat_file')

# get volume
get_volume = pe.Node(fsl.ImageStats(op_string = '-V'), name='get_volume')
get_volume.inputs.in_file = template_brain_mask


# Cluster threshold
grf_cluster = pe.MapNode(fsl.Cluster(), iterfield=['dlh', 'in_file'], name='grf_cluster')
grf_cluster.iterables = [("threshold", [2.3, 3.1])]
grf_cluster.inputs.out_localmax_txt_file = True
grf_cluster.inputs.pthreshold = 0.05
grf_cluster.inputs.out_index_file = True
grf_cluster.inputs.out_threshold_file = True


def volume_convert(input):
    return int(input[0])

workflow.connect(get_volume, ('out_stat', volume_convert), grf_cluster, 'volume')
workflow.connect(smoothestimate, 'dlh', grf_cluster, 'dlh')
workflow.connect(flameo, 'zstats', grf_cluster, 'in_file')

In [8]:
## Datasink
datasink = pe.Node(nio.DataSink(), name='sinker')
datasink.inputs.base_directory=os.path.join(project_folder, 'derivatives', "glm_feat", "group_level_model", "ses-rlsat")


# substitutions = [(f't1w/level2_{stat_type}s/_model_n_{model_n_}_smoothing_fwhm_{fwhm}_space_T1w_subject_id_{sub}/_flameo{contrast_n}/{stat_type}1.nii.gz',
#                   f'sub-{sub}/ses-{ses}/func/fwhm-{fwhm}/model-{model_n_}/sub-{sub}_ses-{ses}_task-{task}_space-T1w_model-{model_n_}_contrast-{contrast_n}_desc-{stat_type}.nii.gz')
#                   for sub in subject_ids
#                   for contrast_n in range(len(contrasts))
#                   for model_n_ in model_n
#                   for fwhm in smoothing_fwhm
#                   for stat_type in ['cope', 'zstat', 'varcope', 'tdof_t']
#                   ]


#grf_thresholded_zstats_file/_contrast_n_0_fwhm_1p5_model_n_0a-z/_run_mode_flame12

substitutions_regexp = [(r'third_level_model/grf_thresholded_zstats_file/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/(\S+)(\d)_threshold.nii.gz',
                         'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\8_flame-\\4_desc-\\7_voxelthreshold-\\5.nii.gz'),
                        (r'third_level_model/level3_.*/_contrast_n_(\d+)_fwhm_(\S{3})_model_n_(\S+)/_run_mode_flame(\d+)/(\S+)(\d).nii.gz',
                          'model-\\3/model-\\3_fwhm-\\2_subjectlevelcontrast-\\1_grouplevelcontrast-\\6_flame-\\4_desc-\\5.nii.gz'),
                        # (r'third_level_model/grf_localmax_.*/fwhm-(\S{3})/model-(\S+)/contrast-(\d+)/_run_mode_flame(\d+)/_threshold_(\S+)/_grf_cluster(\d)/zstat1_(\S+).txt',
                        #   'model-\\2/model-\\2_fwhm-\\1_subjectlevelcontrast-\\3_grouplevelcontrast-\\6_flame-\\4_desc-zstat_\\7-voxelthreshold-\\5.txt')
                       ]

# substitutions = [(f'_contrast_n_{contrast_n}_fwhm_{fwhm}_model_n_{model_n}',
#                   f'fwhm-{fwhm}/model-{model_n}/contrast-{contrast_n}') 
#                  for contrast_n in contrasts
#                  for model_n in model_ns
#                  for fwhm in fwhms]
datasink.inputs.regexp_substitutions = substitutions_regexp


# workflow.connect(copemerge, 'merged_file', datasink, 'copes_merged')
# workflow.connect(level3model, 'design_con', datasink, 'design_con')
# workflow.connect(level3model, 'design_mat', datasink, 'design_mat')

## todo: substitutions
workflow.connect(flameo, 'zstats', datasink, 'third_level_model.level3_zstats')
workflow.connect(flameo, 'copes', datasink, 'third_level_model.level3_copes')
workflow.connect(flameo, 'var_copes', datasink, 'third_level_model.level3_varcopes')
workflow.connect(flameo, 'tdof', datasink, 'third_level_model.level3_tdof_ts')

## cluster results
workflow.connect(grf_cluster, 'threshold_file', datasink, 'third_level_model.grf_thresholded_zstats_file')
workflow.connect(grf_cluster, 'localmax_txt_file', datasink, 'third_level_model.grf_localmax_txt_file')
workflow.connect(grf_cluster, 'index_file', datasink, 'third_level_model.grf_cluster_indices')

In [ ]:
workflow.run(plugin='MultiProc', plugin_args={'n_procs': 22})

241120-15:09:14,122 nipype.workflow INFO:
	 Workflow feat_level3_rlsat settings: ['check', 'execution', 'logging', 'monitoring']
241120-15:09:14,332 nipype.workflow INFO:
	 Running in parallel.
241120-15:09:15,21 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 453.15/453.15, Free processors: 22/22.
241120-15:09:15,117 nipype.workflow INFO:
	 [Job 0] Cached (feat_level3_rlsat.subject_id_getter).
241120-15:09:15,119 nipype.workflow INFO:
	 [Job 1] Cached (feat_level3_rlsat.get_volume).
241120-15:09:15,194 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_rlsat.selector" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_rlsat/_contrast_n_0_fwhm_1p5_model_n_1-dm-RTdur/selector".
241120-15:09:15,195 nipype.workflow INFO:
	 [Node] Setting-up "feat_level3_rlsat.selector" in "/home/Public/trondheim/processing/nipype_workflow_folders/feat_level3_rlsat/_contrast_n_1_fwhm_1p5_model_n_1-dm-RTdur/selector".
241120-15:09:15,195 ni

In [1]:
# !jupyter nbconvert --to script ./07c_GLM-FEAT-learning-tasks-group.ipynb

[NbConvertApp] Converting notebook ./07c_GLM-FEAT-learning-tasks-group.ipynb to script
[NbConvertApp] Writing 14707 bytes to 07c_GLM-FEAT-learning-tasks-group.py


In [11]:
# # subject_ids = ['%02d' % i for i in np.arange(1, 20)]

# templates = {'level2_cope':'/home/gdholla1/projects/bias/data/derivatives/glm_fits_level2/level2_copes/_mask_{mask}_subject_id_{subject_id}/_fwhm_{fwhm}/_shift_{shift}/_flameo{contrast}/cope1.nii.gz',
#              'level2_varcope':'/home/gdholla1/projects/bias/data/derivatives/glm_fits_level2/level2_varcopes/_mask_{mask}_subject_id_{subject_id}/_fwhm_{fwhm}/_shift_{shift}/_flameo{contrast}/varcope1.nii.gz',
#              'level2_tdof':'/home/gdholla1/projects/bias/data/derivatives/glm_fits_level2/level2_tdof/_mask_{mask}_subject_id_{subject_id}/_fwhm_{fwhm}/_shift_{shift}/_flameo{contrast}/tdof_t1.nii.gz',             
#              'epi2struct':'/home/gdholla1/projects/bias/data/derivatives/registration/epi2t1weighted/epi2structmat_ants/_subject_id_{subject_id}/transformComposite.h5',
#              't1weighted':'/home/gdholla1/projects/bias/data/raw/sub-{subject_id}/anat/sub-{subject_id}_T1w.nii.gz',
#              'struct2mni':'/home/gdholla1/projects/bias/data/derivatives/registration/epi2t1weighted/struct2mnimat_ants/_subject_id_{subject_id}/transformComposite.h5',
#              'mask':fsl.Info.standard_image('MNI152_T1_1mm_brain_mask.nii.gz')}

In [12]:
# # Copes at level 2:
# #

# # In case you have only one cope at level 2, specify it on the input anyway.
# #Cope number from level 1
# lvl1_cope=int(sys.argv[1])
# #Cope number from level 2
# lvl2_cope=int(sys.argv[2])


# # The directory where we can find the level 2 output
# l12_out_dir=os.path.join(project_dir,"level2s","model"+model_id)

# wf = pe.Workflow(name='wf')
# wf.base_dir = os.path.join(work_dir,"wdir"+str(model_id)+"lvl3","copes_"+str(lvl1_cope)+"_"+str(lvl2_cope))
# wf.config = {"execution": {"crashdump_dir":os.path.join(project_dir,'crashdumps')}}